<a href="https://colab.research.google.com/github/VladShajdulin/OTUS/blob/main/home_work_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

In [99]:
low, high = -10, 10
n_size = 20000

torch.manual_seed(142)
X = (high - low) * torch.rand((n_size, 2), dtype=torch.float64) + low
y = torch.sin(X[:,0] + 2 * X[:,1]) * torch.exp(-(2 * X[:,0] + X[:,1]) ** 2)
y = torch.unsqueeze(y, -1)

In [100]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=142)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=142)

print(f'Training samples: {X_train.size()[0]}')
print(f'Validating samples: {X_val.size()[0]}')
print(f'Training samples: {X_test.size()[0]}')

Training samples: 14000
Validating samples: 3000
Training samples: 3000


In [101]:
class XY_dataset(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return X.size()[0]

  def __getitem__(self, idx):
    return X[idx], y[idx]

In [102]:
def train(dataloader, model, loss_fn, optimizer, device=device):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)

      # Compute prediction error
      pred = model(X)
      loss = loss_fn(pred, y)

      # Backpropagation
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      if batch % 10 == 0:
          loss, current = loss.item(), (batch + 1) * len(X)
          print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def val(X, y, model, loss_fn):
  model.eval()
  with torch.no_grad():
    pred = model(X)
    loss = loss_fn(pred, y).item()

  print(f"\n Val loss: {loss:>7f} \n")

In [103]:
model_1 = nn.Sequential(
    nn.Linear(2, 10, dtype=torch.float64),
    nn.LeakyReLU(),
    nn.Linear(10, 5, dtype=torch.float64),
    nn.LeakyReLU(),
    nn.Linear(5, 1, dtype=torch.float64)
)

In [104]:
batch_size = 256

train_loader = XY_dataset(X_train, y_train)
train_loader = DataLoader(train_loader, shuffle=True, batch_size=batch_size)
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else 'cpu'
print('Device', device)

Device cpu


In [106]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model_1.parameters(), lr=5e-5)
epochs = 15

for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train(train_loader, model_1, loss_fn, optimizer)
  val(X_val, y_val, model_1, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.048410  [  256/20000]
loss: 0.042024  [ 2816/20000]
loss: 0.047585  [ 5376/20000]
loss: 0.046550  [ 7936/20000]
loss: 0.038611  [10496/20000]
loss: 0.036878  [13056/20000]
loss: 0.039830  [15616/20000]
loss: 0.049744  [18176/20000]

 Val loss: 0.037723 

Epoch 2
-------------------------------
loss: 0.047640  [  256/20000]
loss: 0.040932  [ 2816/20000]
loss: 0.034402  [ 5376/20000]
loss: 0.040704  [ 7936/20000]
loss: 0.027192  [10496/20000]
loss: 0.028488  [13056/20000]
loss: 0.029500  [15616/20000]
loss: 0.047069  [18176/20000]

 Val loss: 0.032756 

Epoch 3
-------------------------------
loss: 0.047744  [  256/20000]
loss: 0.030959  [ 2816/20000]
loss: 0.040355  [ 5376/20000]
loss: 0.031383  [ 7936/20000]
loss: 0.025409  [10496/20000]
loss: 0.030280  [13056/20000]
loss: 0.040428  [15616/20000]
loss: 0.021819  [18176/20000]

 Val loss: 0.029460 

Epoch 4
-------------------------------
loss: 0.029190  [  256/20000]
loss: 0.022238  [ 281

In [107]:
resid = y_test - model_1(X_test)

In [111]:
abs(resid).mean() / abs(y_test).mean()

tensor(2.7032, dtype=torch.float64, grad_fn=<DivBackward0>)

In [113]:
resid / y_test

tensor([[-5.5257e+108],
        [-2.2207e+139],
        [  8.1820e-01],
        ...,
        [  2.9229e+01],
        [ -6.1672e+66],
        [  5.3086e+17]], dtype=torch.float64, grad_fn=<DivBackward0>)